# Molecular set representation learning - Molecular property prediction

## Imports

In [ ]:
from multiprocessing import cpu_count

import torch

import pandas as pd
import lightning.pytorch as pl

from torch.utils.data import DataLoader

from molsetrep.models import (
    LightningSRRegressor,
    LightningDualSRRegressor,
    LightningSRGNNRegressor,
)
from molsetrep.encoders import SingleSetEncoder, DualSetEncoder, GraphEncoder

## Prepare the data

### Load from file

In [ ]:
df_train = pd.read_csv("../data/adme/ADME_MDR1_ER_train.csv")
df_test = pd.read_csv("../data/adme/ADME_MDR1_ER_test.csv")

### Encode the data

#### Single-set (MSR1)

In [ ]:
enc_single = SingleSetEncoder()
dataset_train_single = enc_single.encode(df_train["smiles"], df_train["activity"], torch.float32)
dataset_test_single = enc_single.encode(df_test["smiles"], df_test["activity"], torch.float32)

# Get the dimensions of the encoding
dims_single = [dataset_train_single[0][0].shape[0]]

#### Dual-set (MSR2)

In [ ]:
enc_dual = DualSetEncoder()
dataset_train_dual = enc_dual.encode(df_train["smiles"], df_train["activity"], torch.float32)
dataset_test_dual = enc_dual.encode(df_test["smiles"], df_test["activity"], torch.float32)

# Get the dimensions of the encoding
dims_dual = [
    len(dataset_train_dual[0][0][0]),
    len(dataset_train_dual[0][1][0])
]

#### Set-enhanced GNN (SR-GNN)

In [ ]:
enc_graph = GraphEncoder()
dataset_train_graph = enc_graph.encode(df_train["smiles"], df_train["activity"], label_dtype=torch.float32)
dataset_test_graph = enc_graph.encode(df_test["smiles"], df_test["activity"], label_dtype=torch.float32)

# Get the dimensions of the encoding
dims_graph = [
    dataset_train_graph.dataset[0].num_node_features,
    dataset_train_graph.dataset[0].num_edge_features,
]

### Get torch data loaders

#### Single-set (MSR1)

In [ ]:
train_loader_single = DataLoader(
    dataset_train_single,
    batch_size=64,
    shuffle=True,
    num_workers=cpu_count() if cpu_count() < 8 else 8,
    drop_last=True
)

test_loader_single = DataLoader(
    dataset_test_single,
    batch_size=64,
    shuffle=True,
    num_workers=cpu_count() if cpu_count() < 8 else 8,
    drop_last=True
)

#### Dual-set (MSR2)

In [ ]:
train_loader_dual = DataLoader(
    dataset_train_dual,
    batch_size=64,
    shuffle=True,
    num_workers=cpu_count() if cpu_count() < 8 else 8,
    drop_last=True
)

test_loader_dual = DataLoader(
    dataset_test_dual,
    batch_size=64,
    shuffle=True,
    num_workers=cpu_count() if cpu_count() < 8 else 8,
    drop_last=True
)

#### Set-enhanced GNN (SR-GNN)

In [ ]:
train_loader_graph = dataset_train_graph
test_loader_graph = dataset_test_graph

## Train

### Initialise the model

#### Single-set (MSR1)

In [ ]:
model_single = LightningSRRegressor([64], [8], dims_single)

#### Dual-set (MSR2)

In [ ]:
model_dual = LightningDualSRRegressor([64, 64], [8, 8], dims_dual)

#### Set-enhanced GNN (SR-GNN)

In [ ]:
model_graph = LightningSRGNNRegressor(
    [128, 128], [64, 64],
    n_hidden_channels=[128, 64],
    n_in_channels=dims_graph[0], 
    n_edge_channels=dims_graph[1],
    n_layers=8
)

### Initialise the trainer and fit

#### Single-set (MSR1)

In [ ]:
trainer_single = pl.Trainer(
    max_epochs=250,
)

trainer_single.fit(model_dual, train_dataloaders=train_loader_single)
trainer_single.test(dataloaders=test_loader_single)

#### Dual-set (MSR2)

In [ ]:
trainer_dual = pl.Trainer(
    max_epochs=250,
)

trainer_dual.fit(model_dual, train_dataloaders=train_loader_dual)
trainer_dual.test(dataloaders=test_loader_dual)

#### Set-enhanced GNN (SR-GNN)

In [ ]:
trainer_graph = pl.Trainer(
    max_epochs=900,
)

trainer_graph.fit(model_graph, train_dataloaders=train_loader_graph)
trainer_graph.test(dataloaders=test_loader_graph)